In [2]:
# important imports
import numpy as np
import math
import itertools
import time
from multiprocessing import Pool
import scipy.sparse
from IPython.display import clear_output
from multiprocessing.dummy import Pool as ThreadPool
import csv
import ast
import networkx as nx

In [3]:
#Given an omino (nxn binary matrix), checks if it is valid
# i.e. all ones are rook-adjacent and contiguous

def valid_omino(omino):
    '''
    takes an omino as input and validates it
    returns a boolean
    '''
    
    k = omino.shape[0]
    check = np.zeros([k,k]) - omino
    
    
    _f  = False
    for i in range(k):
        for j in range(k):
            if check[i,j] == -1:
                if not _f:
                    _f  = True
                    check[i,j] = 1
    if not _f:
        print("ABORT")
        return
    
    # I know this should require fewer passes, but...
    for iter in range(k**2):
        for i in range(k):
            for j in range(k):
                if check[i,j] == 1:
                    if i != 0:
                        check[i-1,j] = check[i-1,j]**2

                    if i != k-1:
                        check[i+1,j] = check[i+1,j]**2

                    if j != 0:
                        check[i,j-1] = check[i,j-1]**2

                    if j != k-1:
                        check[i,j+1] = check[i,j+1]**2
                    
    return np.sum(check) == np.sum(omino)


# given a list of (r,c) index pairs, generates an omino
# if the omino isn't valid, returns the 0x0 0 matrix
def make_omino(inds, dim):
    '''
    takes a set of indices (x,y) to be set to 1
    makes a matrix of size dim x dim
    returns the matrix if it is a valid omino
    returns the empty zeros matrix otherwise
    
    '''
    p = np.zeros([dim,dim])
    for loc in inds:
        p[loc[0],loc[1]] = 1
    
    if valid_omino(p):
        return p
    else:
        return np.zeros([0,0])

    

# generates all of the <cells>-ominos in the <grid>^2 grid
def make_omino_set(cells, grid):
    '''
    takes two numbers, cells and grid and returns a list
    of all the valid cells-ominos which fit in the
    grid^2 grid
    
    returns a list of valid ominos    
    '''
    
    if grid % cells != 0:
        print("WARNING - <cells> doesnt divide <grid>")
        #return []
    
    
    pair_idx = []
    for i in range(grid):
        for j in range(grid):
            pair_idx.append((i,j))
    
    
    ominos = []
    
    for t in (l for l in itertools.combinations(pair_idx, cells)):
        p = make_omino(t,grid)
        if p.shape[0] != 0:
            ominos.append(p)
    
    return ominos


In [4]:
# the queen contiguity versions of the above functions
# checks if an omino is queen-contiguity valid
def valid_omino_qc(omino):
    k = omino.shape[0]
    check = np.zeros([k,k]) - omino
    
    
    _f  = False
    for i in range(k):
        for j in range(k):
            if check[i,j] == -1:
                if not _f:
                    _f  = True
                    check[i,j] = 1
    if not _f:
        print("ABORT")
        return
    
    for iter in range(15):
        for i in range(k):
            for j in range(k):
                if check[i,j] == 1:
                    if i != 0:
                        check[i-1,j] = check[i-1,j]**2


                    if i != k-1:
                        check[i+1,j] = check[i+1,j]**2

                    if j != 0:
                        check[i,j-1] = check[i,j-1]**2

                    if j != k-1:
                        check[i,j+1] = check[i,j+1]**2
                        
                    if i!= 0 and j!=0:
                        check[i-1,j-1] = check[i-1,j-1]**2
                        
                    if i!=0 and j!= k-1:
                        check[i-1,j+1] = check[i-1,j+1]**2
                        
                    if i != k-1 and j != 0:
                        check[i+1,j-1] = check[i+1,j-1]**2
                   
                    if j != k-1 and i != 0:
                        check[i-1,j+1] = check[i-1,j+1]**2
                        
                    if i!= k-1 and j != k-1:
                        check[i+1,j+1] = check[i+1,j+1]**2
                    
                    
    return np.sum(check) == np.sum(omino)



# given a list of (r,c) index pairs, generates a queen
# contiguous omino
# if the omino isn't valid, returns the 0x0 0 matrix
def make_omino_qc(inds, dim):
    dim
    p = np.zeros([dim,dim])
    for loc in inds:
        p[loc[0],loc[1]] = 1
    
    if valid_omino_qc(p):
        return p
    else:
        return np.zeros([0,0])
    
    
    
#makes queen continguous 'ominos'
def make_omino_set_qc(cells, grid):
   
    if grid % cells != 0:
        print("WARNING - <cells> doesn't divide <grid>")
        #return []
    
    
    pair_idx = []
    for i in range(grid):
        for j in range(grid):
            pair_idx.append((i,j))
    
    
    ominos = []
    
    for t in (l for l in itertools.combinations(pair_idx, cells)):
        p = make_omino_qc(t,grid)
        if p.shape[0] != 0:
            ominos.append(p)
    
    return ominos
        

In [5]:
# since matrices don't hash, here's how this all works:
# 1. generate the set of legal ominos
# 2. make a list of indices into that lis
# 3. generate all of the tuples of indices
#    which correspond to valid pairs, triples, quads,
#    etc. of these ominos
#    a tuple is "valid" if none of them overlap
# 4. use these tuples to index back into the set
#    of ominos


# declare some variables to make later stuff work:
pairs_inds = []
trips_inds = []
quads_inds = []
inds = []

def make_trips(inx):
    '''
    takes in an index and returns the valid triples
    of ominos which include that index
    
    designed to be used with a .map(make_trips, inds) call 
    
    '''
    val_trips = []
    counter = 0
    global pairs_inds
    for d in pairs_inds:
        c=inx
        counter +=1
        #if counter%10000 == 0: print(counter/10000)
        q = set(d+(c,))
        if len(q) == 3 and c>max(d):
            check  = sum([n[i] for i in q])
            if 2 not in check: val_trips.append(tuple(q))
    #print("returning on ",inx)
    return val_trips

def make_quads(inx):
    '''
    takes in an index and returns the valid four-tuples
    of ominos which include that index
    
    designed to be used with a .map(make_quads, inds) call 
    
    '''
    val_quads = []
    counter = 0
    global trips_inds
    for t in trips_inds:
        c=inx
        counter +=1
        #if counter%10000 == 0: print(counter/10000)
        p = set(t+(c,))
        if len(p) == 4 and c>max(t):
            check  = sum([n[i] for i in p])
            if 2 not in check: val_quads.append(tuple(p))
    #if inx%10==0: print("returning on ",inx)
    return val_quads




def make_pents(inx):
    '''
    takes in an index and returns the valid five-tuples
    of ominos which include that index
    
    designed to be used with a .map(make_pents, inds) call 
    
    '''
    val_pents = []
    counter = 0
    global quads_inds
    for t in quads_inds:
        c=inx
        counter +=1
        #if counter%10000 == 0: print(counter/10000)
        p = set(t+(c,))
        if len(p) == 5 and c>max(t):
            check  = sum([n[i] for i in p])
            if 2 not in check: val_pents.append(tuple(p))
    #if inx%10==0: print("returning on ",inx)
    return val_pents

In [ ]:
# this box gives the partitions of the 4x4 grid into
# 4 regions of size 4

n = make_omino_set(4,4)
print(len(n))
print([np.sum(x) for x in n if np.sum(x) != 4])
inds = range(len(n))
pairs_inds = [x for x in itertools.combinations(inds,2) if 2 not in n[x[0]]+n[x[1]]]

# deploy some threads to make the valid triples
pool = Pool(8)
results = pool.map(make_trips,inds)
trips_inds  = [item for sublist in results for item in sublist]
trips_inds = list(set([tuple(sorted(j)) for j in trips_inds]))
pool = Pool(8)

# we can throw out a bunch of these triples:
ones = np.ones([4,4])
trips_inds = [ t for t in trips_inds if valid_omino(ones-sum([n[i] for i in t]))]

# and now the fours
results = pool.map(make_quads,inds)
quads_inds = [item for sublist in results for item in sublist]
quads_inds = list(set([tuple(sorted(j)) for j in quads_inds]))

print(len(quads_inds))


partitions = [[n[i].astype(int).tolist() for i in p] for p in quads_inds]



# this block will do saving

with open('parts_4-4_rc.csv','w') as f:
    wr = csv.writer(f)
    wr.writerows(partitions)

    
# this block is how you read it back in
with open('parts_4-4_rc.csv','r') as f:
    reader = csv.reader(f)
    l = list(reader)
        
partitions = [[ast.literal_eval(t) for t in a] for a  in l]

In [ ]:
# this box gives the partitions of the 4x4 grid into
# 4 regions of size 4 - queen contiguous

n = make_omino_set_qc(4,4)
print(len(n))
print([np.sum(x) for x in n if np.sum(x) != 4])
inds = range(len(n))
pairs_inds = [x for x in itertools.combinations(inds,2) if 2 not in n[x[0]]+n[x[1]]]

# threads for threes
pool = Pool(8)
results = pool.map(make_trips,inds)
trips_inds  = [item for sublist in results for item in sublist]
trips_inds = list(set([tuple(sorted(j)) for j in trips_inds]))
pool = Pool(8)

# we can throw out a bunch of these triples:
ones = np.ones([4,4])
trips_inds = [ t for t in trips_inds if valid_omino_qc(ones-sum([n[i] for i in t]))]


# and for fours
results = pool.map(make_quads,inds)
quads_inds = [item for sublist in results for item in sublist]
quads_inds = list(set([tuple(sorted(j)) for j in quads_inds]))

print(len(quads_inds))


partitions = [[n[i].astype(int).tolist() for i in p] for p in quads_inds]



# this block will do saving

with open('parts_4-4_qc.csv','w') as f:
    wr = csv.writer(f)
    wr.writerows(partitions)

    
# this block is how you read it back in
with open('parts_4-4_qc.csv','r') as f:
    reader = csv.reader(f)
    l = list(reader)
        
partitions = [[ast.literal_eval(t) for t in a] for a  in l]

In [ ]:
# this box gives the partitions of the 4x4 grid into
# 4 regions of size either 3, 4, or 5

n = make_omino_set_qc(3,4)
n += make_omino_set_qc(4,4)
n += make_omino_set_qc(5,4)
print(len(n))
inds = range(len(n))
pairs_inds = [x for x in itertools.combinations(inds,2) if 2 not in n[x[0]]+n[x[1]]]

# deploy some threads to make the valid triples
pool = Pool(8)
results = pool.map(make_trips,inds)
trips_inds  = [item for sublist in results for item in sublist]
trips_inds = list(set([tuple(sorted(j)) for j in trips_inds]))
pool = Pool(8)

# we can throw out a bunch of these triples:
ones = np.ones([4,4])
trips_inds = [ t for t in trips_inds if valid_omino(ones-sum([n[i] for i in t]))]

# and now the fours
results = pool.map(make_quads,inds)
quads_inds = [item for sublist in results for item in sublist]
quads_inds = list(set([tuple(sorted(j)) for j in quads_inds]))

print(len(quads_inds))


partitions = [[n[i].astype(int).tolist() for i in p] for p in quads_inds]



# this block will do saving

with open('parts_4-4_345_qc.csv','w') as f:
    wr = csv.writer(f)
    wr.writerows(partitions)

    
# this block is how you read it back in
with open('parts_4-4_345_rc.csv','r') as f:
    reader = csv.reader(f)
    l = list(reader)
        
partitions = [[ast.literal_eval(t) for t in a] for a  in l]

WARNING - <cells> doesn't divide <grid>
WARNING - <cells> doesn't divide <grid>
1707


In [6]:
# and finally the 5x5 grid into 5 pieces of size 5, rook contiguous


n = make_omino_set(5,5)
print(len(n))
inds = range(len(n))
pairs_inds = [x for x in itertools.combinations(inds,2) if 2 not in n[x[0]]+n[x[1]]]

# deploy some threads to make the valid triples
pool = Pool(8)
results = pool.map(make_trips,inds)
trips_inds  = [item for sublist in results for item in sublist]
trips_inds = list(set([tuple(sorted(j)) for j in trips_inds]))



# and now the fours
pool = Pool(8)
results = pool.map(make_quads,inds)
quads_inds = [item for sublist in results for item in sublist]
quads_inds = list(set([tuple(sorted(j)) for j in quads_inds]))

571


In [7]:
# we can throw out a bunch of these fours:
ones = np.ones([5,5])
#quads_inds = [ q for q in quads_inds if valid_omino(ones-sum([n[i] for i in q]))]

# and now the fives
pool = Pool(8)
results = pool.map(make_pents,inds)
pents_inds = [item for sublist in results for item in sublist]
pents_inds = list(set([tuple(sorted(j)) for j in pents_inds]))

print(len(pents_inds))


np.save("pents_inds_55_rc", pents_inds)

partitions = [[n[i].astype(int).tolist() for i in p] for p in pents_inds]



# this block will do saving

with open('parts_5-5_rc.csv','w') as f:
    wr = csv.writer(f)
    wr.writerows(partitions)

  
# this block is how you read it back in
with open('parts_5-5_rc.csv','r') as f:
    reader = csv.reader(f)
    l = list(reader)
        
partitions = [[ast.literal_eval(t) for t in a] for a  in l]

4006


In [5]:
print(len(quads_inds))

20030


In [5]:
l = nx.laplacian_spectrum(G).tolist()
l.sort()
print(l[1])
l = nx.laplacian_spectrum(H).tolist()
l.sort()
print(l[1])

NameError: name 'G' is not defined

In [30]:
n = []
for i in range(7):
    n+= make_omino_set_qc(i+1,3)
print(len(n))
inds=list(range(len(n)))
ones = np.ones([3,3])

trips_inds = [x for x in itertools.combinations(inds,3) if 0 not in n[x[0]]+n[x[1]]+n[x[2]]  and 2 not in n[x[0]]+n[x[1]]+n[x[2]]]  

print(len(trips_inds))
partitions = [[n[i].astype(int).tolist() for i in p] for p in trips_inds]

with open('parts_1-7-3_qc.csv','w') as f:
    wr = csv.writer(f)
    wr.writerows(partitions)

print(len(partitions))   
# this block is how you read it back in
with open('parts_3-3_rc.csv','r') as f:
    reader = csv.reader(f)
    l = list(reader)
        
partitions = [[ast.literal_eval(t) for t in a] for a  in l]

WARNING - <cells> doesn't divide <grid>
WARNING - <cells> doesn't divide <grid>
WARNING - <cells> doesn't divide <grid>
WARNING - <cells> doesn't divide <grid>
WARNING - <cells> doesn't divide <grid>
378
17938
17938


In [10]:
n = make_omino_set(2,3)
n += make_omino_set(3,3)
n += make_omino_set(4,3)
print(len(n))
inds=list(range(len(n)))
ones = np.ones([3,3])

trips_inds = [x for x in itertools.combinations(inds,3) if np.array_equiv(ones,n[x[0]]+n[x[1]]+n[x[2]])]  

print(len(trips_inds))
partitions = [[n[i].astype(int).tolist() for i in p] for p in trips_inds]

with open('parts_234-3_rc_fixed.csv','w') as f:
    wr = csv.writer(f)
    wr.writerows(partitions)

print(len(partitions))   
# this block is how you read it back in
with open('parts_234-3_qc.csv','r') as f:
    reader = csv.reader(f)
    l = list(reader)
        
partitions = [[ast.literal_eval(t) for t in a] for a  in l]l = [g for g in df['geometry'] if not g isinstance(shapely.geometry.multipolygon.MultiPolygon) or not g isinstance(shapely.geometry.polygon.Polygon)]


WARNING - <cells> doesnt divide <grid>
WARNING - <cells> doesnt divide <grid>
70
58
58


In [ ]:
l = [g for g in df['geometry'] if not g isinstance(shapely.geometry.multipolygon.MultiPolygon) or not g isinstance(shapely.geometry.polygon.Polygon)]
